In [ ]:
import os

# Setup Paths

### Change Model Name And URL As Needed

In [ ]:
# You Can Change CUSTOM_MODEL_NAME To Whatever You Want Your Transfer Learning Model To Be Saved As
# You Can Change PRETRAINED_MODEL_NAME and PRETRAINED_MODEL_URL To Another Pretrained Model From Tensorflow at: github.com/tensorflow/models/tree/master/research/object_detection/models
# Both Can Stay As Is And It Will Work Fine
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

### Creates Paths To Directories Specified In Paths Dictionary

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

##### 

# 1. Download Pretrained Models from Tensorflow Model Zoo And Install Tensorflow Object Detection

In [ ]:
# https://www.tensorflow.org/install/source_windows

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

### Clones Tensorflow Model Garden To: Tensorflow > Models (Note: Not Tensorflow > Workspace > Models)

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

### You need Google Protocol Buffers Installed
### For Mac OS I Found Using Brew Instead of PIP Only Worked For Protobuf
### Feel Free To Modify Code As Needed Depending On Your OS

In [ ]:
# For Mac OS
!brew install protobuf-compiler

### Install Tensorflow Object Detection From The Repo You Just Cloned

In [ ]:
if os.name=='posix':  
#   !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python3 -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

### Script To Verify If Tensorflow Object Detection Is Installed Correctly
### Errors Can Usually Be Fixed By Installing A Module Or Uninstalling And Reinstalling Module
### You Want To See "OK (skipped=1)" On The Last Line

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
# Common Problem Fixes
# !pip install tensorflow --upgrade
# !pip uninstall protobuf matplotlib -y
# !pip install protobuf matplotlib==3.2

In [ ]:
# If You Have Trouble Importing Object_Detection You May Need To Restart The Kernel On This Notebook

In [ ]:
import object_detection

### Clones Tensorflow Pretrained Model Into Pre-Trained-Models In Our Workspace Directory 
### Located At: Tensorflow > Workspace > Pre-Trained-Models
### We Use This To Create The Files We Need

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

##### 

# 2. Create Label Map

### Creates Label Map File In: Tensorflow > Workspace > Annotations
### Change Labels To Match Your Labels Set In Notebook "1-Image-Collection" Step 2
### The Case And Spelling Should Be The Same You Used To Label Objects In LabeImg Software
### Example: My Labels Where ['thumbsup', 'thumbsdown'] But In LabelImg I Used "ThumbsUp" For thumbsup Label And "ThumbsDown" For thumbsdown Label So I Use That Case Here

In [ ]:
labels = [{'name':'ThumbsUp', 'id':1}, {'name':'ThumbsDown', 'id':2}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

##### 

# 3. Create TF records

### Generates Files In Correct Format To Train From 
### Uses Your Annotated Images(Image and XML Files) You Saved Then Runs Them Through The Script

### OPTIONAL IF RUNNING ON COLAB

In [ ]:
# ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
# if os.path.exists(ARCHIVE_FILES):
#   !tar -zxvf {ARCHIVE_FILES}

### Clones Script Into: Tensorflow > Scripts

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

### Creates test.record And train.record File From Your Images and XML Files
### Located With The Label Map File At: Tensorflow > Workspace > Annotations

In [ ]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

##### 

# 4. Copy Model Config To Training Folder

### This Clones The pipeline.config File From The Pre-Trained-Model Directory At: Tensorflow > Workspace > Pre-Trained-Models > PRETRAINED_MODEL_NAME
### Pastes It Into Our Training Models Directory In: Tensorflow > Workspace > Models > CUSTOM_MODEL_NAME

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

##### 

# 5. Update Config For Transfer Learning

### Update The pipeline.config File We Just Put Into Our Models Directory With The Correct Info

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = len(labels)
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

### Check To Make Sure Your Pipeline.Config File Has Changed
### Sometimes You Have To Run Through Step 5 Again

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
config

###### 

# 6. Train The Model

In [ ]:
# May Need Pip Install tensorflow=={version number} tensorflow-gpu=={version number}
# If You Want To Use Your GPU Look UP How To Install CUDA And cuDNN 

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

### Run In Terminal To See Progress
### If You Run This Command And It Gets Stuck Delete All The Files In Tensorflow > Workspace > Annotations (label_map.pbtxt, test.record, train.record)
### Restart This Notebook And Run Through The Entire Notebook Again
### If You See "Use fn_output_signature instead" On The last Line In Terminal Wait 5 Min To See If It Starts To Show Steps Runnig. This Is What You Want To See.
### If You See Something Else You Likely Need To Run Through This Notebook Again

In [ ]:
# Training Steps Can Be Changed, But 2,000 Is A Good Start
print(command)

In [ ]:
# !{command}

# You Now Have Your Model Trained

## Go To Notebook "3-Run-Object-Detection-Model" To Test Your Model On Image Or Webcam
## Or
## Evaluate Model Metrics Below


# 

# Evaluate the Model

### Run This In Terminal To See Model Metrics

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

### Run In Terminal To See Progress

In [ ]:
print(command)

In [ ]:
#!{command}

# 

# Evaluate With TensorBoard

### In Terminal Go To Your Trained Model Directory At: Tensorflow > Workspace > Models > YOUR-MODEL-NAME > train
### OR
### In Terminal Go To Your Trained Model Directory At: Tensorflow > Workspace > Models > YOUR-MODEL-NAME > eval

### Run The Following Command In Either Train Or Eval Directory To Evaluate With TensorBoard


In [ ]:
tensorboard --logdir=.

# 